In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

In [39]:
from pyspark.sql.functions import *

In [64]:
df=spark.read.format("csv").option("header","true").option("inferschema","true").load("/FS.csv");
df.show()

+-------+---+------+---------+
|Country|Age|Salary|Purchased|
+-------+---+------+---------+
| France| 44| 72000|       No|
|  Spain| 27| 48000|      Yes|
|Germany| 30| 54000|       No|
|  Spain| 38| 61000|       No|
|Germany| 40|  1000|      Yes|
| France| 35| 58000|      Yes|
|  Spain| 78| 52000|       No|
| France| 48| 79000|      Yes|
|Germany| 50| 83000|       No|
| France| 37| 67000|      Yes|
+-------+---+------+---------+



In [65]:
df.count(),len(df.columns)

(10, 4)

In [66]:
df.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Purchased: string (nullable = true)



In [67]:
df1=df.withColumn("Purchased_Index",when(df['Purchased'] =='Yes',1).otherwise(0)).drop('Purchased');

In [68]:
df1.show()

+-------+---+------+---------------+
|Country|Age|Salary|Purchased_Index|
+-------+---+------+---------------+
| France| 44| 72000|              0|
|  Spain| 27| 48000|              1|
|Germany| 30| 54000|              0|
|  Spain| 38| 61000|              0|
|Germany| 40|  1000|              1|
| France| 35| 58000|              1|
|  Spain| 78| 52000|              0|
| France| 48| 79000|              1|
|Germany| 50| 83000|              0|
| France| 37| 67000|              1|
+-------+---+------+---------------+



In [69]:
df1.columns

['Country', 'Age', 'Salary', 'Purchased_Index']

In [70]:
ks1=(df1.select(mean('Age').alias('Mean_Value')).collect()[0].asDict()['Mean_Value'])
gs1=(df1.select(stddev_pop('Age').alias('StandardDeviation_Value')).collect()[0].asDict()['StandardDeviation_Value'])

In [71]:
ks2=(df1.select(mean('Salary').alias('Mean_Value')).collect()[0].asDict()['Mean_Value'])
gs2=(df1.select(stddev_pop('Salary').alias('StandardDeviation_Value')).collect()[0].asDict()['StandardDeviation_Value'])

In [72]:
df1.withColumn('Standardized_Age',(df['Age']-ks1)/gs1).withColumn('Standardized_Salary',(df['Salary']-ks2)/gs2).show()

+-------+---+------+---------------+-------------------+--------------------+
|Country|Age|Salary|Purchased_Index|   Standardized_Age| Standardized_Salary|
+-------+---+------+---------------+-------------------+--------------------+
| France| 44| 72000|              0|0.09536934736019054|  0.6652706149031185|
|  Spain| 27| 48000|              1|-1.1517682719653808| -0.4358669545916983|
|Germany| 30| 54000|              0|-0.9316851626726329| -0.1605825622179941|
|  Spain| 38| 61000|              0|-0.3447968712253052|  0.1605825622179941|
|Germany| 40|  1000|              1|-0.1980747983634733| -2.5922613615190477|
| France| 35| 58000|              1|-0.5648799805180531|0.022940366031142013|
|  Spain| 78| 52000|              0| 2.5896445860113335|-0.25234402634256214|
| France| 48| 79000|              1| 0.3888134930838544|  0.9864357393391066|
|Germany| 50| 83000|              0| 0.5355355659456863|  1.1699586675882427|
| France| 37| 67000|              1|-0.4181579076562212|  0.4358

In [73]:
ks1=(df1.select(min('Age').alias('Min_Value')).collect()[0].asDict()['Min_Value'])
gs1=(df1.select(max('Age').alias('Max_Valeu')).collect()[0].asDict()['Max_Valeu'])

In [74]:
ks2=(df1.select(min('Salary').alias('Min_Value')).collect()[0].asDict()['Min_Value'])
gs2=(df1.select(max('Salary').alias('Max_Valeu')).collect()[0].asDict()['Max_Valeu'])

In [75]:
df1.withColumn('Normalized',(df['Age']-ks1)/(gs1-ks1)).withColumn('Normalizeda',(df['Salary']-ks2)/(gs2-ks2)).show()

+-------+---+------+---------------+--------------------+------------------+
|Country|Age|Salary|Purchased_Index|          Normalized|       Normalizeda|
+-------+---+------+---------------+--------------------+------------------+
| France| 44| 72000|              0|  0.3333333333333333|0.8658536585365854|
|  Spain| 27| 48000|              1|                 0.0| 0.573170731707317|
|Germany| 30| 54000|              0|0.058823529411764705|0.6463414634146342|
|  Spain| 38| 61000|              0| 0.21568627450980393|0.7317073170731707|
|Germany| 40|  1000|              1|  0.2549019607843137|               0.0|
| France| 35| 58000|              1|  0.1568627450980392|0.6951219512195121|
|  Spain| 78| 52000|              0|                 1.0|0.6219512195121951|
| France| 48| 79000|              1|  0.4117647058823529|0.9512195121951219|
|Germany| 50| 83000|              0| 0.45098039215686275|               1.0|
| France| 37| 67000|              1| 0.19607843137254902|0.8048780487804879|